<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# **扩展 Tiktoken BPE 分词器，添加新 Token**  

- 本笔记本介绍 **如何扩展现有的 BPE 分词器**，并重点讲解 **如何在 OpenAI 的 [Tiktoken](https://github.com/openai/tiktoken) 实现中添加新 Token**。  
- 如果需要 **分词的基础知识**，请参考 [第 2 章](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb) 和 **BPE from Scratch** [教程](link)。  
- 例如，假设我们有一个 **GPT-2 分词器**，并希望对以下文本进行编码：

In [1]:
import tiktoken

base_tokenizer = tiktoken.get_encoding("gpt2")
sample_text = "Hello, MyNewToken_1 is a new token. <|endoftext|>"

token_ids = base_tokenizer.encode(sample_text, allowed_special={"<|endoftext|>"})
print(token_ids)

[15496, 11, 2011, 3791, 30642, 62, 16, 318, 257, 649, 11241, 13, 220, 50256]


- **遍历每个 Token ID**，可以帮助我们更好地理解 **如何通过词汇表解码 Token ID**：  

In [2]:
for token_id in token_ids:
    print(f"{token_id} -> {base_tokenizer.decode([token_id])}")

15496 -> Hello
11 -> ,
2011 ->  My
3791 -> New
30642 -> Token
62 -> _
16 -> 1
318 ->  is
257 ->  a
649 ->  new
11241 ->  token
13 -> .
220 ->  
50256 -> <|endoftext|>


- 如上所示，**"MyNewToken_1" 被拆分为 5 个子词 Token**，这对于 BPE 处理 **未知词汇** 时是正常行为。  
- 但如果 **"MyNewToken_1" 是一个特殊 Token**，我们希望它像 **`"<|endoftext|>"`** 一样 **作为单个 Token 进行编码**，本笔记本将讲解如何实现该功能。  

&nbsp;
## 1. 添加特殊的token

- 需要注意，我们必须 **将新 Token 作为特殊 Token 添加**。原因在于：  
  - **新 Token 在原始分词器训练过程中并未出现**，因此 **没有对应的“合并规则”（merges）**。  
  - 即使我们手动创建这些合并规则，**也很难在不破坏现有分词体系的情况下，将其正确整合**（详情请参考 **BPE from Scratch** 笔记本 [链接] 了解“合并规则”）。  

- 例如，假设我们希望 **添加 2 个新 Token**：

In [3]:
# Define custom tokens and their token IDs
custom_tokens = ["MyNewToken_1", "MyNewToken_2"]
custom_token_ids = {
    token: base_tokenizer.n_vocab + i for i, token in enumerate(custom_tokens)
}

- 接下来，我们创建一个自定义的 **`Encoding`** 对象，用于存储 **特殊 Token**，具体如下：  

In [4]:
# Create a new Encoding object with extended tokens
extended_tokenizer = tiktoken.Encoding(
    name="gpt2_custom",
    pat_str=base_tokenizer._pat_str,
    mergeable_ranks=base_tokenizer._mergeable_ranks,
    special_tokens={**base_tokenizer._special_tokens, **custom_token_ids},
)

- 就这样！现在我们可以验证 **分词器是否能够正确编码示例文本**：  

- 如我们所见，**新添加的 Token**（`50257` 和 `50258`）**已成功编码到输出中**：  

In [5]:
special_tokens_set = set(custom_tokens) | {"<|endoftext|>"}

token_ids = extended_tokenizer.encode(
    "Sample text with MyNewToken_1 and MyNewToken_2. <|endoftext|>",
    allowed_special=special_tokens_set
)
print(token_ids)

[36674, 2420, 351, 220, 50257, 290, 220, 50258, 13, 220, 50256]


- 同样，我们还可以 **逐个 Token 检查编码结果**：  

In [6]:
for token_id in token_ids:
    print(f"{token_id} -> {extended_tokenizer.decode([token_id])}")

36674 -> Sample
2420 ->  text
351 ->  with
220 ->  
50257 -> MyNewToken_1
290 ->  and
220 ->  
50258 -> MyNewToken_2
13 -> .
220 ->  
50256 -> <|endoftext|>


- 如上所示，我们已成功 **更新分词器**。  
- 但如果要将其用于 **预训练的 LLM**，还需要 **更新 LLM 的嵌入层（embedding layer）和输出层（output layer）**，具体方法将在 **下一节** 进行讲解。  

&nbsp;
## 2. 更新预训练的LLM

- 本节将讲解 **如何在更新分词器后，对现有的预训练 LLM 进行相应调整**。  
- 我们将使用 **书中主章节所采用的原始预训练 GPT-2 模型** 进行演示。  

&nbsp;
### 2.1 加载预训练的GPT

In [7]:
# Relative import from the gpt_download.py contained in this folder
from gpt_download import download_and_load_gpt2

settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

checkpoint: 100%|███████████████████████████| 77.0/77.0 [00:00<00:00, 34.4kiB/s]
encoder.json: 100%|███████████████████████| 1.04M/1.04M [00:00<00:00, 4.78MiB/s]
hparams.json: 100%|█████████████████████████| 90.0/90.0 [00:00<00:00, 24.7kiB/s]
model.ckpt.data-00000-of-00001: 100%|███████| 498M/498M [00:33<00:00, 14.7MiB/s]
model.ckpt.index: 100%|███████████████████| 5.21k/5.21k [00:00<00:00, 1.05MiB/s]
model.ckpt.meta: 100%|██████████████████████| 471k/471k [00:00<00:00, 2.33MiB/s]
vocab.bpe: 100%|████████████████████████████| 456k/456k [00:00<00:00, 2.45MiB/s]


In [8]:
# Relative import from the gpt_download.py contained in this folder
from previous_chapters import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

# Define model configurations in a dictionary for compactness
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# Copy the base configuration and update with specific model settings
model_name = "gpt2-small (124M)"  # Example model name
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})

gpt = GPTModel(NEW_CONFIG)
gpt.eval();

&nbsp;
### 2.2 使用预训练过的GPT

- 接下来，我们使用 **原始分词器** 和 **更新后的分词器** 对以下示例文本进行编码，并进行对比：  

In [9]:
sample_text = "Sample text with MyNewToken_1 and MyNewToken_2. <|endoftext|>"

original_token_ids = base_tokenizer.encode(
    sample_text, allowed_special={"<|endoftext|>"}
)

In [10]:
new_token_ids = extended_tokenizer.encode(
    "Sample text with MyNewToken_1 and MyNewToken_2. <|endoftext|>",
    allowed_special=special_tokens_set
)

- 现在，我们将 **原始 Token ID 输入到 GPT 模型中**：  

In [11]:
import torch

with torch.no_grad():
    out = gpt(torch.tensor([original_token_ids]))

print(out)

tensor([[[ 0.2204,  0.8901,  1.0138,  ...,  0.2585, -0.9192, -0.2298],
         [ 0.6745, -0.0726,  0.8218,  ..., -0.1768, -0.4217,  0.0703],
         [-0.2009,  0.0814,  0.2417,  ...,  0.3166,  0.3629,  1.3400],
         ...,
         [ 0.1137, -0.1258,  2.0193,  ..., -0.0314, -0.4288, -0.1487],
         [-1.1983, -0.2050, -0.1337,  ..., -0.0849, -0.4863, -0.1076],
         [-1.0675, -0.5905,  0.2873,  ..., -0.0979, -0.8713,  0.8415]]])


- 如上所示，模型能够正常运行 **（为简洁起见，代码仅显示原始输出，未将其转换回文本）**。  
- 若需了解 **如何将模型输出转换回文本**，请参考 **第 5 章 [链接] 的 `generate` 函数（5.3.3 节）**。  

- 如果我们 **使用更新后的分词器生成的 Token ID** 再次输入模型，会发生什么情况？  

```python
with torch.no_grad():
    gpt(torch.tensor([new_token_ids]))

print(out)

...
# IndexError: index out of range in self
```

- 如我们所见，这会导致 **索引错误（Index Error）**。  
- 这是因为 **GPT 模型的输入嵌入层（Embedding Layer）和输出层（Output Layer）** 预设了固定的 **词汇表大小（Vocabulary Size）**，而更新后的分词器可能已超出该范围：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/extend-tiktoken/gpt-updates.webp" width="400px">

&nbsp;
### **2.3 更新嵌入层（Updating the Embedding Layer）**  

- 我们首先 **更新模型的嵌入层（Embedding Layer）**。  
- 首先，需要注意 **嵌入层包含 50,257 个条目**，这正好对应于 **原始词汇表的大小**：  

In [12]:
gpt.tok_emb

Embedding(50257, 768)

- 我们希望 **扩展嵌入层**，**增加 2 个新 Token**。  
- 简而言之，我们 **创建一个更大的嵌入层**，然后 **将原始嵌入层的权重复制到新嵌入层中**。  

In [13]:
num_tokens, emb_size = gpt.tok_emb.weight.shape
new_num_tokens = num_tokens + 2

# Create a new embedding layer
new_embedding = torch.nn.Embedding(new_num_tokens, emb_size)

# Copy weights from the old embedding layer
new_embedding.weight.data[:num_tokens] = gpt.tok_emb.weight.data

# Replace the old embedding layer with the new one in the model
gpt.tok_emb = new_embedding

print(gpt.tok_emb)

Embedding(50259, 768)


- 如上所示，我们的 **嵌入层（Embedding Layer）已成功扩展**。  

&nbsp;
### **2.4 更新输出层（Updating the Output Layer）**  

- 接下来，我们需要 **扩展输出层（Output Layer）**，该层当前包含 **50,257 个输出特征**，其大小与嵌入层的词汇表大小相同。  
- **（顺带一提，你可能会对额外的学习资料感兴趣，其中探讨了 PyTorch 中 `Linear` 层与 `Embedding` 层的相似性。）**  

In [14]:
gpt.out_head

Linear(in_features=768, out_features=50257, bias=False)

- **扩展输出层（Output Layer）的过程** 与 **扩展嵌入层（Embedding Layer）** 类似：  

In [15]:
original_out_features, original_in_features = gpt.out_head.weight.shape

# Define the new number of output features (e.g., adding 2 new tokens)
new_out_features = original_out_features + 2

# Create a new linear layer with the extended output size
new_linear = torch.nn.Linear(original_in_features, new_out_features)

# Copy the weights and biases from the original linear layer
with torch.no_grad():
    new_linear.weight[:original_out_features] = gpt.out_head.weight
    if gpt.out_head.bias is not None:
        new_linear.bias[:original_out_features] = gpt.out_head.bias

# Replace the original linear layer with the new one
gpt.out_head = new_linear

print(gpt.out_head)

Linear(in_features=768, out_features=50259, bias=True)


- 首先，我们先 **使用原始 Token ID 测试模型**，观察其是否仍能正常运行： 

In [16]:
with torch.no_grad():
    output = gpt(torch.tensor([original_token_ids]))
print(output)

tensor([[[ 0.2267,  0.9132,  1.0494,  ..., -0.2330, -0.3008, -1.1458],
         [ 0.6808, -0.0495,  0.8574,  ...,  0.0671,  0.5572, -0.7873],
         [-0.1947,  0.1045,  0.2773,  ...,  1.3368,  0.8479, -0.9660],
         ...,
         [ 0.1200, -0.1027,  2.0549,  ..., -0.1519, -0.2096,  0.5651],
         [-1.1920, -0.1819, -0.0981,  ..., -0.1108,  0.8435, -0.3771],
         [-1.0612, -0.5674,  0.3229,  ...,  0.8383, -0.7121, -0.4850]]])


- 接下来，让我们 **测试更新后的模型在新增 Token 上的表现**。   

In [17]:
with torch.no_grad():
    output = gpt(torch.tensor([new_token_ids]))
print(output)

tensor([[[ 0.2267,  0.9132,  1.0494,  ..., -0.2330, -0.3008, -1.1458],
         [ 0.6808, -0.0495,  0.8574,  ...,  0.0671,  0.5572, -0.7873],
         [-0.1947,  0.1045,  0.2773,  ...,  1.3368,  0.8479, -0.9660],
         ...,
         [-0.0656, -1.2451,  0.7957,  ..., -1.2124,  0.1044,  0.5088],
         [-1.1561, -0.7380, -0.0645,  ..., -0.4373,  1.1401, -0.3903],
         [-0.8961, -0.6437, -0.1667,  ...,  0.5663, -0.5862, -0.4020]]])


- 如我们所见，**模型已成功支持扩展后的 Token 集**。  
- 实际应用中，我们通常需要 **对模型进行微调（Fine-tuning）或持续预训练（Continual Pretraining）**，特别是 **新扩展的嵌入层（Embedding Layer）和输出层（Output Layer）**，以确保模型能够有效学习新 Token 的表示。  

### **关于权重共享（Weight Tying）**  

- **如果模型使用了权重共享（Weight Tying）**，即 **嵌入层（Embedding Layer）与输出层（Output Layer）共享相同的权重**（类似于 **Llama 3** [链接]），那么 **扩展输出层的过程将更为简单**。  
- 在这种情况下，我们 **只需直接将嵌入层的权重复制到输出层**：  

In [18]:
gpt.out_head.weight = gpt.tok_emb.weight

In [19]:
with torch.no_grad():
    output = gpt(torch.tensor([new_token_ids]))